In [1]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install --upgrade --force-reinstall llama-cpp-python --no-cache-dir

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.9/36.9 MB 100.8 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 KB 214.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 KB 262.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 94.2 MB/s eta 0:00:00a 0:00:01
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.56-cp310-cp310-manylinux_2_35_x86_64.whl size=26407560 sha256=3f7e98d10d1b9abd7f3edcf824d97b0f7d67ee667405aae23fb3004e99d7ff72
  Stored in directory: /tmp/pip-ephem-wheel-cache-w744758z/wheels/e5/09/9d/c413053f6258cb2546cc792418c595e276f9efd5db31a80377
Successfully built llama-cpp-python
  Attempting uninstall: typing-extensions
   

In [9]:
!export LLAMA_CPP_LIB="/home/default/workspace/venv/lib/python3.10/site-packages/llama_cpp/libllama.so"

In [28]:
pip install guardrails-ai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.7/177.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.1/113.1 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 46.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.3/110.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 39.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.3/195.3 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [29]:
from llama_cpp import Llama

In [30]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import LlamaCpp

In [31]:
import os
from langchain.prompts import PromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text, Number
from langchain.output_parsers import PydanticOutputParser
# from gpt3tools import PydanticOutputParser
from pydantic import BaseModel, Field, conint, PositiveInt
from langchain_core.pydantic_v1 import BaseModel, Field, validator, conint, PositiveInt
from enum import Enum
from typing import List, Dict, Optional
from datetime import datetime
from enum import Enum
from langchain.output_parsers import OutputFixingParser
from langchain.schema import OutputParserException
from langchain.schema.runnable import Runnable
from transformers import AutoModelForCausalLM, AutoTokenizer,pipeline
import torch
from langchain_community.llms import HuggingFacePipeline
from langchain.memory import ChatMessageHistory
from langchain_core.messages import AIMessage,HumanMessage,SystemMessage
from langchain.chains import ConversationChain
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

In [32]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [33]:
class Position(str, Enum):
    top_right = "top right"
    top_left = "top left"
    center = "center"
    bottom_right = "bottom right"
    bottom_left = "bottom left"
    center_right = "center right"
    center_top = "center top"

class Structure(str, Enum):
    cannon = "cannon"
    mortar = "mortar"
    tower = "tower"
    headquarters = "headquarters"
    resource = "resource"

class Troops(str,Enum):
    shooters = "shooters"
    tanks = "tanks"
    helicopters = "helicopters"

class Defense(BaseModel):
    structure: Structure = Field(description="A defense structure type. Can have only one Headquarters.")
    position: Position = Field(description="Position of the defense structure.")
    # level: PositiveInt = Field(description="Level of the defense")
    # shotSpeed: Optional[conint(ge=1)] = Field(None, description="Speed of shots (if applicable)")
    # hitpoint: Optional[PositiveInt] = Field(None, description="Hitpoints of the defense")
    # damage: Optional[PositiveInt] = Field(None, description="Damage caused by the defense")

class ShooterLevels(BaseModel):
    health: PositiveInt = Field(description="Health of the shooter")
    damage: PositiveInt = Field(description="Damage of the shooter")
    speed: float = Field(description="Speed of the shooter")

class Shooters(BaseModel):
    noShooters: PositiveInt = Field(description="Number of shooters")
    # shooterMaxLevel: PositiveInt = Field(description="Maximum level of shooters")
    # levels: Dict[str, ShooterLevels] = Field(description="Levels of shooters")

class TankLevels(BaseModel):
    health: PositiveInt = Field(description="Health of the tank")
    damage: PositiveInt = Field(description="Damage of the tank")

class Tanks(BaseModel):
    noTanks: PositiveInt = Field(description="Number of tanks")
    # tanksMaxLevel: PositiveInt = Field(description="Maximum level of tanks")
    # levels: Dict[str, TankLevels] = Field(description="Levels of tanks")

class HelicopterLevels(BaseModel):
    health: PositiveInt = Field(description="Health of the helicopter")
    damage: PositiveInt = Field(description="Damage of the helicopter")

class Helicopters(BaseModel):
    noHelicopters: PositiveInt = Field(description="Number of helicopters")
    # shooterMaxLevel: PositiveInt = Field(description="Maximum level of helicopters")
    # levels: Dict[str, HelicopterLevels] = Field(description="Levels of helicopters")

class Environment(BaseModel):
    defenses: Dict[str, Defense] = Field(description="Dictionary of defenses. Keys of the dictionary will be like 'defense1','defense2' etc.")

class Agents(BaseModel):
    shooters: Shooters = Field(description="Details of shooters")
    tanks: Tanks = Field(description="Details of tanks")
    helicopters: Helicopters = Field(description="Details of helicopters")

class troopDeployment(BaseModel):
    troop: Troops = Field(description = "Type of agent deployed.")
    DeployedNumbers: PositiveInt = Field(description = "Number of agents deployed.")
    DeploymentPosition: Position = Field(description="Position of the agents deployed.")

class Deployment(BaseModel):
        deployments: Dict[str,troopDeployment] = Field(description = "Information about different types of agents deployed and their deployment positions. Keys will be like 'troop1','troop2' etc.")

class GameConfig(BaseModel):
    environment: Environment = Field(description="Game environment configuration, responsible for running the game.")
    agents: Agents = Field(description="Details of agents.")
    deployment: Deployment = Field(description = "Deployment information about the agents.")

parser = PydanticOutputParser(pydantic_object=GameConfig)


In [34]:
schema = GameConfig.schema_json()
new_prompt = ChatPromptTemplate.from_messages(
    messages=[
       SystemMessage(
            content=f"<|im_start|>You are a helpful assistant that answers in JSON. Here's the json schema you must adhere to:\n<schema>\n{schema}\n</schema><|im_end|>"
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template(
            "<|im_start|>{human_input}<|im_end|>"
        ),  # Where the human input will injected
    ],
)

In [35]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
llm = LlamaCpp(model_path = "/home/default/workspace/models/Hermes-2-Pro-Mistral-7B.Q4_K_M.gguf",
                n_gpu_layers = -1,
                n_batch = 1024,
                callback_manager = callback_manager,
                verbose = True,
               n_ctx = 2048,
               max_tokens = 1024,
              )

llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from /home/default/workspace/models/Hermes-2-Pro-Mistral-7B.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = jeffq
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 

In [36]:
chat_llm_chain = LLMChain(
    llm=llm,
    prompt=new_prompt,
    verbose=True,
    memory=memory,
)

In [25]:
query = "Generate a game configuration with six defenses. Of these six defenses, two are mortars placed at the center and center-right position. There are two towers at top-right and top-left corners. Also, two cannons are at the bottom-right and bottom-left positions. There are twenty agents available to us. Of these twenty, ten are shooters, five are tanks, and five are helicopters. I want to deploy five shooters from the top-right corner and five from the bottom-right corner. I want to deploy five tanks from the center-right and five helicopters from the bottom-left position."

chat_llm_chain.predict(human_input = query)



> Entering new LLMChain chain...
Prompt after formatting:
System: <|im_start|>You are a helpful assistant that answers in JSON. Here's the json schema you must adhere to:
<schema>
{"title": "GameConfig", "type": "object", "properties": {"environment": {"title": "Environment", "description": "Game environment configuration, responsible for running the game.", "allOf": [{"$ref": "#/definitions/Environment"}]}, "agents": {"title": "Agents", "description": "Details of agents.", "allOf": [{"$ref": "#/definitions/Agents"}]}, "deployment": {"title": "Deployment", "description": "Deployment information about the agents.", "allOf": [{"$ref": "#/definitions/Deployment"}]}}, "required": ["environment", "agents", "deployment"], "definitions": {"Structure": {"title": "Structure", "description": "An enumeration.", "enum": ["cannon", "mortar", "tower", "headquarters", "resource"], "type": "string"}, "Position": {"title": "Position", "description": "An enumeration.", "enum": ["top right", "top left"

KeyboardInterrupt: 